In [1]:
import pandas as pd

df = pd.read_csv('data/PP-Scrips.csv')

#exclude = ['KPITTECH','AIRTELPP','TATASTLLP','MTARTECH','NAVKARCORP','SEYAIND']

df = df.dropna()

stock_ids = df['NSE ID'].unique()
#stock_ids = df[~df['NSE ID'].isin(exclude) ]['NSE ID'].unique()


In [2]:
stock_ids

array(['BAJAJHLDNG', 'ITC', 'IEX', 'HCLTECH', 'ICICIBANK', 'HEROMOTOCO',
       'AXISBANK', 'POWERGRID', 'CDSL', 'HDFCBANK', 'MCX', 'PERSISTENT',
       'MPHASIS', 'BALKRISIND', 'MOTILALOFS', 'IPCALAB', 'OFSS',
       'DRREDDY', 'SUNPHARMA', 'CADILAHC', 'ICRA', 'LUPIN'], dtype=object)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

BAJAJHLDNG
160
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-1

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
CDSL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[***********

[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
OFSS
162
2019-02-15 00:00:00
[***********

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,BAJAJHLDNG.NS,2020-08-15,2598.85,61.02,90.0,269.72,405.0,9.64
7,BAJAJHLDNG.NS,2020-11-15,2707.10,78.36,116.0,255.65,403.0,10.59
8,BAJAJHLDNG.NS,2021-02-15,3544.60,103.29,100.0,275.14,416.0,12.88
9,BAJAJHLDNG.NS,2021-05-16,3504.20,85.27,92.0,327.94,398.0,10.69
10,BAJAJHLDNG.NS,2021-08-15,4197.50,69.96,98.0,336.88,406.0,12.46
...,...,...,...,...,...,...,...,...
7,LUPIN.NS,2020-11-15,906.25,4.66,3835.0,-2.82,14978.0,-321.37
8,LUPIN.NS,2021-02-15,1053.00,9.66,4017.0,25.28,15226.0,41.65
9,LUPIN.NS,2021-05-16,1178.45,10.15,3783.0,26.83,15163.0,43.92
10,LUPIN.NS,2021-08-15,976.85,11.95,4270.0,36.42,15905.0,26.82


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-4-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-4-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-4-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,BAJAJHLDNG.NS,2020-08-15,2598.85,61.02,90.0,269.72,405.0,9.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BAJAJHLDNG.NS,2020-11-15,2707.10,78.36,116.0,255.65,403.0,10.59,61.02,269.72,90.0,9.64,2598.85,-5.22,9.35,4.13,28.89,28.42,4.13
8,BAJAJHLDNG.NS,2021-02-15,3544.60,103.29,100.0,275.14,416.0,12.88,78.36,255.65,116.0,10.59,2707.10,7.62,23.27,30.89,-13.79,31.81,30.89
9,BAJAJHLDNG.NS,2021-05-16,3504.20,85.27,92.0,327.94,398.0,10.69,103.29,275.14,100.0,12.88,3544.60,19.19,-20.26,-1.07,-8.00,-17.45,-1.07
10,BAJAJHLDNG.NS,2021-08-15,4197.50,69.96,98.0,336.88,406.0,12.46,85.27,327.94,92.0,10.69,3504.20,2.73,17.02,19.75,6.52,-17.95,19.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,LUPIN.NS,2020-11-15,906.25,4.66,3835.0,-2.82,14978.0,-321.37,2.36,-10.29,3528.0,-98.95,1018.20,-72.59,61.60,-10.99,8.70,97.46,-10.99
8,LUPIN.NS,2021-02-15,1053.00,9.66,4017.0,25.28,15226.0,41.65,4.66,-2.82,3835.0,-321.37,906.25,-996.45,1012.65,16.20,4.75,107.30,16.20
9,LUPIN.NS,2021-05-16,1178.45,10.15,3783.0,26.83,15163.0,43.92,9.66,25.28,4017.0,41.65,1053.00,6.13,5.78,11.91,-5.83,5.07,11.91
10,LUPIN.NS,2021-08-15,976.85,11.95,4270.0,36.42,15905.0,26.82,10.15,26.83,3783.0,43.92,1178.45,35.74,-52.85,-17.11,12.87,17.73,-17.11


In [5]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,HEROMOTOCO.NS,2021-11-15,55.17,191.11,-3.22
11,BAJAJHLDNG.NS,2021-11-15,50.00,45.33,18.76
11,MOTILALOFS.NS,2021-11-15,47.39,142.36,17.78
11,CDSL.NS,2021-11-15,24.79,34.53,19.11
11,IEX.NS,2021-11-15,20.88,22.86,93.71
11,DRREDDY.NS,2021-11-15,17.03,161.65,4.10
11,BALKRISIND.NS,2021-11-15,14.92,18.30,3.03
11,PERSISTENT.NS,2021-11-15,9.84,6.92,32.73
11,MPHASIS.NS,2021-11-15,6.61,0.44,21.78
11,ITC.NS,2021-11-15,3.85,13.16,10.12


In [6]:
df_analysis_final.to_csv('data/PP_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_growth = pd.read_csv('data/PP_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,BAJAJHLDNG.NS,2020-11-15,2707.10,78.36,116.0,255.65,403.0,10.59,61.02,269.72,90.0,9.64,2598.85,-5.22,9.35,4.13,28.89,28.42,4.13
2,BAJAJHLDNG.NS,2021-02-15,3544.60,103.29,100.0,275.14,416.0,12.88,78.36,255.65,116.0,10.59,2707.10,7.62,23.27,30.89,-13.79,31.81,30.89
3,BAJAJHLDNG.NS,2021-05-16,3504.20,85.27,92.0,327.94,398.0,10.69,103.29,275.14,100.0,12.88,3544.60,19.19,-20.26,-1.07,-8.00,-17.45,-1.07
4,BAJAJHLDNG.NS,2021-08-15,4197.50,69.96,98.0,336.88,406.0,12.46,85.27,327.94,92.0,10.69,3504.20,2.73,17.02,19.75,6.52,-17.95,19.75
5,BAJAJHLDNG.NS,2021-11-15,4984.30,101.67,147.0,360.19,437.0,13.84,69.96,336.88,98.0,12.46,4197.50,6.92,11.84,18.76,50.00,45.33,18.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,LUPIN.NS,2020-11-15,906.25,4.66,3835.0,-2.82,14978.0,-321.37,2.36,-10.29,3528.0,-98.95,1018.20,-72.59,61.60,-10.99,8.70,97.46,-10.99
128,LUPIN.NS,2021-02-15,1053.00,9.66,4017.0,25.28,15226.0,41.65,4.66,-2.82,3835.0,-321.37,906.25,-996.45,1012.65,16.20,4.75,107.30,16.20
129,LUPIN.NS,2021-05-16,1178.45,10.15,3783.0,26.83,15163.0,43.92,9.66,25.28,4017.0,41.65,1053.00,6.13,5.78,11.91,-5.83,5.07,11.91
130,LUPIN.NS,2021-08-15,976.85,11.95,4270.0,36.42,15905.0,26.82,10.15,26.83,3783.0,43.92,1178.45,35.74,-52.85,-17.11,12.87,17.73,-17.11


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

MOTILALOFS.NS    4
CDSL.NS          3
PERSISTENT.NS    3
IEX.NS           2
DRREDDY.NS       2
BALKRISIND.NS    2
ITC.NS           2
MPHASIS.NS       2
HEROMOTOCO.NS    2
BAJAJHLDNG.NS    2
LUPIN.NS         2
IPCALAB.NS       1
OFSS.NS          1
SUNPHARMA.NS     1
ICRA.NS          1
POWERGRID.NS     1
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
71,PERSISTENT.NS,2021-11-15,9.84,6.92,32.73,0,0,0.0,0.0,0.0
17,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
53,CDSL.NS,2021-11-15,24.79,34.53,19.11,0,0,2.0,3.0,5.0
89,MOTILALOFS.NS,2021-11-15,47.39,142.36,17.78,0,0,4.0,4.0,8.0
107,DRREDDY.NS,2021-11-15,17.03,161.65,4.10,0,0,3.0,6.0,9.0
5,BAJAJHLDNG.NS,2021-11-15,50.00,45.33,18.76,0,0,5.0,7.0,12.0
35,HEROMOTOCO.NS,2021-11-15,55.17,191.11,-3.22,0,0,NaN,NaN,NaN
83,BALKRISIND.NS,2021-11-15,14.92,18.30,3.03,0,0,NaN,NaN,NaN


In [11]:
df_growth[df_growth['Stock'] == 'BAJAJHLDNG.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
1,BAJAJHLDNG.NS,2020-11-15,28.89,28.42,4.13
2,BAJAJHLDNG.NS,2021-02-15,-13.79,31.81,30.89
3,BAJAJHLDNG.NS,2021-05-16,-8.00,-17.45,-1.07
4,BAJAJHLDNG.NS,2021-08-15,6.52,-17.95,19.75
5,BAJAJHLDNG.NS,2021-11-15,50.00,45.33,18.76
